<h1>Analyzing CIA Factbook Data Using SQL</h1>

In this project, we'll work with data from the <a id="https://www.cia.gov/library/publications/the-world-factbook/">CIA World Factbook</a>, a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like:

- population - The population as of 2015.
- population_growth - The annual population growth rate, as a percentage.
- area - The total land and water area.

<h3>Connecting Jupyter Notebook to our Database File</h3>

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

<h3>Description of Tables in the Database</h3>

In [5]:
%%sql
SELECT * FROM sqlite_master WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


<h3>First 5 rows of the Facts Table in the Database</h3>

In [6]:
%%sql
SELECT * FROM facts limit 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns:

- name - The name of the country.
- area - The total land and sea area of the country.
- population - The country's population.
- population_growth- The country's population growth as a percentage.
- birth_rate - The country's birth rate, or the number of births a year per 1,000 people.
- death_rate - The country's death rate, or the number of death a year per 1,000 people.
- area- The country's total area (both land and water).
- area_land - The country's land area in square kilometers.
- area_water - The country's waterarea in square kilometers.

<h3>Total Number Of Rows In Facts Table</h3>

In [34]:
%%sql
SELECT COUNT(*) AS ROWS FROM facts;

Done.


ROWS
261


<h3>Summary Statistics of Facts Table</h3>

In [8]:
%%sql
SELECT MIN(population) AS MIN_POPULATION,
        MAX(population) AS MAX_POPULATION,
    MIN(population_growth) AS MIN_POPULATION_GROWTH,
    MAX(population_growth) AS MAX_POPULATION_GROWTH FROM facts;

Done.


MIN_POPULATION,MAX_POPULATION,MIN_POPULATION_GROWTH,MAX_POPULATION_GROWTH
0,7256490011,0.0,4.02


<h3>Outlier Countries</h3>

It seems like the table contains a row for the whole world, which explains the population of over 7.2 billion. It also seems like the table contains a row for Antarctica, which explains the population of 0.<br> This seems to match the CIA Factbook<a id="cia.gov/library/publications/the-world-factbook/geos/ay.html"> page for Antarctica:</a>

In [14]:
%%sql
SELECT name,population FROM facts where population=(SELECT MIN(population) FROM facts);

Done.


name,population
Antarctica,0


In [15]:
%%sql
SELECT name,population FROM facts where population=(SELECT MAX(population) FROM facts);

Done.


name,population
World,7256490011


<h3>Exploring Average population and Average Area.</h3>

In [16]:
%%sql
SELECT AVG(population) as AVG_POPULATION,
AVG(area) AS AVG_AREA FROM facts;

Done.


AVG_POPULATION,AVG_AREA
62094928.32231405,555093.546184739


<h3>Finding countries that are densely populated. </h3>

In [17]:
%%sql
SELECT * FROM facts
WHERE population > (
    SELECT AVG(population)
    FROM facts
)
  AND area < (
    SELECT AVG(area)
    FROM facts
)

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
173,th,Thailand,513120,510890,2230,67976405,0.34,11.19,7.8,0.0
185,uk,United Kingdom,243610,241930,1680,64088222,0.54,12.17,9.35,2.54
192,vm,Vietnam,331210,310070,21140,94348835,0.97,15.96,5.93,0.3


Bangladesh,Germany,Japan,Philippines,Thailand,United Kingdom, Vietnam are the densely populated countries as of 2015 stated in Central Intelligence Agency(CIA) Factbook Data.

<h3>Finding countries that are sparsely populated.</h3>

In [18]:
%%sql
SELECT * FROM facts
WHERE population < (
    SELECT AVG(population)
    FROM facts
)
  AND area > (
    SELECT AVG(area)
    FROM facts
)

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46
7,ar,Argentina,2780400,2736690,43710,43431886,0.93,16.64,7.33,0.0
9,as,Australia,7741220,7682300,58920,22751014,1.07,12.15,7.14,5.65
21,bl,Bolivia,1098581,1083301,15280,10800882,1.56,22.76,6.52,0.62
23,bc,Botswana,581730,566730,15000,2182719,1.21,20.96,13.39,4.56
28,bm,Burma,676578,653508,23070,56320206,1.01,18.39,7.96,0.28
32,ca,Canada,9984670,9093507,891163,35099836,0.75,10.28,8.42,5.66
34,ct,Central African Republic,622984,622984,0,5391539,2.13,35.08,13.8,0.0


These are the sparsely populated countries as of 2015 stated in Central Intelligence Agency(CIA) Factbook Data.

<h3>Countries having more water than land</h3>

In [20]:
%%sql
SELECT * FROM facts WHERE area_water>area_land;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
228,io,British Indian Ocean Territory,54400,60,54340,None,None,None,None,None
247,vq,Virgin Islands,1910,346,1564,103574,0.59,10.31,8.54,7.67


<h3>Countries having highest ratios of water to land</h3>

In [29]:
%%sql
SELECT * from facts
WHERE ((area_water*1000)/area_land)>(SELECT AVG((area_water*1000)/area_land) FROM facts);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
228,io,British Indian Ocean Territory,54400,60,54340,None,None,None,None,None
247,vq,Virgin Islands,1910,346,1564,103574,0.59,10.31,8.54,7.67


<h3>Countries having higher death rate than birth rate</h3>

In [23]:
%%sql
SELECT name, death_rate,birth_rate FROM facts WHERE death_rate>birth_rate;

Done.


name,death_rate,birth_rate
Austria,9.42,9.41
Belarus,13.36,10.7
Bosnia and Herzegovina,9.75,8.87
Bulgaria,14.44,8.92
Croatia,12.18,9.45
Czech Republic,10.34,9.63
Estonia,12.4,10.51
Germany,11.42,8.47
Greece,11.09,8.66
Hungary,12.73,9.16


<h3> Countries which will add the most people to their population next year</h3>

In [30]:
%%sql
SELECT * from facts
WHERE ((birth_rate*1000)/death_rate)>(SELECT AVG((birth_rate*1000)/death_rate) FROM facts);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46
13,ba,Bahrain,760,760,0,1346613,2.41,13.66,2.69,13.09
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
18,bh,Belize,22966,22806,160,347369,1.87,24.68,5.97,0.0
19,bn,Benin,112622,110622,2000,10448647,2.78,36.02,8.21,0.0
21,bl,Bolivia,1098581,1083301,15280,10800882,1.56,22.76,6.52,0.62
25,bx,Brunei,5765,5265,500,429646,1.62,17.32,3.52,2.43
27,uv,Burkina Faso,274200,273800,400,18931686,3.03,42.03,11.72,0.0
29,by,Burundi,27830,25680,2150,10742276,3.28,42.01,9.27,0.0
